In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hariomnagar123/face-mask-detection-dataset")

print("Path to dataset files:", path)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Dropout,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix,classification_report
import seaborn as sns

In [ ]:
path='/root/.cache/kagglehub/datasets/hariomnagar123/face-mask-detection-dataset/versions/1'
print(os.listdir(path))

In [ ]:
path=path+'/FMD_Dataset'
print(os.listdir(path))

In [ ]:
train=path+"/train"
valid=path+"/valid"


In [ ]:
train_gen=ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2
)

valid_gen=ImageDataGenerator(
    rescale=1./255
)

train_data=train_gen.flow_from_directory(
    train,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary'
)

valid_data=valid_gen.flow_from_directory(
    valid,
    target_size=(128,128),
    batch_size=32,
    class_mode='binary',
    shuffle=False 
)

In [ ]:
labels=train_data.class_indices
class_weights=compute_class_weight(
    class_weight="balanced",
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights_dict=dict(enumerate(class_weights))
print(class_weights_dict)

In [ ]:
model=Sequential([
    Conv2D(32,(3,3),activation='relu',input_shape=(128,128,3)),
    MaxPooling2D((2,2)),

    Conv2D(64,(3,3),activation='relu'),
    MaxPooling2D((2,2)),

    Conv2D(128,(3,3),activation='relu'),
    MaxPooling2D((2,2)),

    GlobalAveragePooling2D(),
    Dense(128,activation='relu'),
    Dropout(0.5),
    Dense(1,activation='sigmoid')


])

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
early_stop=EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
    )

reduce_lr=ReduceLROnPlateau(
    monitor='val_loss',
    patience=2,
    factor=0.5,
    min_lr=1e-6
)

In [ ]:
history=model.fit(
    train_data,
    epochs=25,
    validation_data=valid_data,
    class_weight=class_weights_dict,
    callbacks=[early_stop,reduce_lr]
)

In [ ]:
val_loss,val_acc=model.evaluate(valid_data)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_acc}")

In [ ]:
valid_data.reset()
y_pred=model.predict(valid_data,verbose=1)
y_pred=(y_pred>0.5).astype(int).flatten()
y_true=valid_data.classes
labels=list(valid_data.class_indices.keys())

cm=confusion_matrix(y_true,y_pred)

plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=labels, yticklabels=labels)
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.savefig("confusion_matrix.png")  

print("\nClassification Report:\n")
print(classification_report(y_true, y_pred, target_names=labels))